In [1]:
import os

os.environ['MLFLOW_TRACKING_URI']= 'https://dagshub.com/ganu0811/datascienceproject.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']= 'ganu0811'
os.environ['MLFLOW_TRACKING_PASSWORD']= '408b0d3ca30244534b813dfd774bda4b64cc1013'


In [2]:
%pwd

'd:\\MLFlow\\datascienceproject\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\MLFlow\\datascienceproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str   

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
                
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)
                
                create_directories([self.config.artifacts_root])
                
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        params= self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path= config.test_data_path,
            model_path = config. model_path,
            all_params = params,
            metric_file_name= config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = 'https://dagshub.com/ganu0811/datascienceproject.mlflow'
        )
        
    
        return model_evaluation_config
            

In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
from urllib.parse import urlparse
import mlflow.sklearn
import numpy as np
import joblib

In [15]:
class ModelEvaluation:
    
    def __init__(self, config=ModelEvaluationConfig):
        
        self.config = config
    
    def eval_metrics(self, actual, pred):
        
        rmse = np.sqrt(mean_absolute_error(actual, pred))
        mse = mean_squared_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mse, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        
        with mlflow.start_run():
            
            predicted_qualities = model.predict(test_x)
            
            (rmse, mse, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving the log/metric parameters
            metrics_file = Path(self.config.metric_file_name)
            metrics_file.parent.mkdir(parents=True, exist_ok=True)
            metrics_file.touch(exist_ok=True)
            scores = {'rmse': rmse, 'mse': mse, 'r2': r2}
            save_json(path = Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric('rmse',rmse)
            mlflow.log_metric('mse',mse)
            mlflow.log_metric('r2',r2)
            
            # Model registry does not work with file store
            
            if tracking_url_type_store != 'file':
                
                # Register the mode
                # There are other ways to use the model registry, which depends on the use case,
                
                mlflow.sklearn.log_model(model, "model", registered_model_name= "ElasticNetModel")
            
            else:
                mlflow.sklearn.log_model(model, 'model')
                 
            
            
        
        

In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-12-30 16:49:34,879: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-30 16:49:34,883: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-30 16:49:34,889: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-30 16:49:34,891: INFO: common: directory created at artifacts]
[2024-12-30 16:49:34,893: INFO: common: directory created at artifacrs/model_evaluation]
[2024-12-30 16:49:35,134: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/12/30 16:49:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2024/12/30 16:50:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run useful-fish-469 at: https://dagshub.com/ganu0811/datascienceproject.mlflow/#/experiments/0/runs/829657cc3d174914a4e7d62aa20e71df
🧪 View experiment at: https://dagshub.com/ganu0811/datascienceproject.mlflow/#/experiments/0
